In [1]:
import pandas as pd
import os
import numpy as np
# from IPython.display import display, Markdown

# Import the required ArcGIS API for Python modules
import arcgis
from arcgis.gis import GIS
gis = arcgis.GIS()

In [2]:
# directories
working_directory   = os.getcwd()
data_folder         = os.path.join(working_directory, "data"         )
intermediate_folder = os.path.join(working_directory, "intermediate" )
results_folder      = os.path.join(working_directory, "results"      )
webapplayers_folder = os.path.join(results_folder   , r"webapplayers" )
webapplayersshps_folder = os.path.join(results_folder   , r"webapplayers\shapefiles" )
webapplayerszips_folder = os.path.join(results_folder   , r"webapplayers\shapefiles_zips" )

dfModes = pd.DataFrame({'mode'      :['ALL','RAI' ,'BUS','BIC'    ,'PED'       ,'TRU'  ],
                        'mode_label':['All','Rail','Bus','Bicycle','Pedestrian','Truck'],
                        'mode_char' :['A'  ,'R'   , 'S' ,'K'      ,'P'         ,'T'    ]})
display(dfModes)

csvODWithGroups = os.path.join(intermediate_folder,'od_by_mode_with_groups.csv')

# show numbers with commas
pd.options.display.float_format = '{:,.4f}'.format

,mode,mode_label,mode_char
0,ALL,All,A
1,RAI,Rail,R
2,BUS,Bus,S
3,BIC,Bicycle,K
4,PED,Pedestrian,P
5,TRU,Truck,T


In [3]:
dfODWithGroups = pd.read_csv(csvODWithGroups)
display(dfODWithGroups)

dfODWithGroupsMode = pd.DataFrame.merge(dfODWithGroups, dfModes, on='mode')
dfODWithGroupsMode = dfODWithGroupsMode.drop(columns=(['mode_label','mode']))
dfODWithGroupsMode = dfODWithGroupsMode.rename(columns={'mode_char':'mode'})
display(dfODWithGroupsMode)

,OID,DID,day_type,day_part,odvolume,odtime,mode,date_range,group
0,101,101,0,0,15720,770.0000,TRU,2019_01,smalldist
1,101,101,0,1,984,812.0000,TRU,2019_01,smalldist
2,101,101,0,2,3093,881.0000,TRU,2019_01,smalldist
3,101,101,0,3,7330,743.0000,TRU,2019_01,smalldist
4,101,101,0,4,3056,703.0000,TRU,2019_01,smalldist
...,...,...,...,...,...,...,...,...,...
12739480,662,662,2,0,15,"1,199.0000",ALL,2021_YR,smalldist
12739481,662,662,2,2,5,NaN,ALL,2021_YR,smalldist
12739482,662,662,2,3,4,NaN,ALL,2021_YR,smalldist
12739483,662,662,2,4,3,NaN,ALL,2021_YR,smalldist


,OID,DID,day_type,day_part,odvolume,odtime,date_range,group,mode
0,101,101,0,0,15720,770.0000,2019_01,smalldist,T
1,101,101,0,1,984,812.0000,2019_01,smalldist,T
2,101,101,0,2,3093,881.0000,2019_01,smalldist,T
3,101,101,0,3,7330,743.0000,2019_01,smalldist,T
4,101,101,0,4,3056,703.0000,2019_01,smalldist,T
...,...,...,...,...,...,...,...,...,...
12739480,423,424,1,4,7,261.0000,2019_04,smalldist,R
12739481,336,361,0,0,3,"2,136.0000",2019_10,smalldist,R
12739482,336,361,0,3,3,"2,136.0000",2019_10,smalldist,R
12739483,336,361,1,0,5,"2,136.0000",2019_10,smalldist,R


In [4]:
# CREATE DATASET THAT HAS TOTAL ORIGIN AND DESTINATION VOLUME FOR EACH CENTER/SMALL DIST

# unique key fields for origin and destination volumes
ogrpflds = ['OID','day_type','day_part','mode','date_range','group']
dgrpflds = ['DID','day_type','day_part','mode','date_range','group']

dfOrigGroup = dfODWithGroupsMode.groupby(ogrpflds,as_index=False).agg(OVOL=('odvolume','sum'))
dfDestGroup = dfODWithGroupsMode.groupby(dgrpflds,as_index=False).agg(DVOL=('odvolume','sum'))
display(dfOrigGroup)
display(dfDestGroup)

# outer join on O-D
dfODGroup = pd.DataFrame.merge(dfOrigGroup, dfDestGroup, left_on=ogrpflds, right_on=dgrpflds, how='outer')

# fill na's with 0, meaning there was no record
dfODGroup['OVOL'] = dfODGroup['OVOL'].fillna(0)
dfODGroup['DVOL'] = dfODGroup['DVOL'].fillna(0)

dfODGroup.loc[dfODGroup['OID'].isna(), 'OID'] = dfODGroup['DID']
dfODGroup.loc[dfODGroup['DID'].isna(), 'DID'] = dfODGroup['OID']

dfODbyID = dfODGroup.drop(columns=('DID'))
dfODbyID = dfODbyID.rename(columns=({'OID':'ID'}))
dfODbyID['BVOL'] = dfODbyID['OVOL'] + dfODbyID['DVOL']
dfODbyID


,OID,day_type,day_part,mode,date_range,group,OVOL
0,11,0,0,A,2019_01,centertosmalldist,53
1,11,0,0,A,2019_01,smalldist,852
2,11,0,0,A,2019_02,centertosmalldist,54
3,11,0,0,A,2019_02,smalldist,823
4,11,0,0,A,2019_03,centertosmalldist,60
...,...,...,...,...,...,...,...
643821,2014,2,5,T,2020_BM,centertosmalldist,129
643822,2014,2,5,T,2020_YR,centertosmalldist,98
643823,2014,2,5,T,2021_06,centertosmalldist,180
643824,2014,2,5,T,2021_BM,centertosmalldist,15


,DID,day_type,day_part,mode,date_range,group,DVOL
0,11,0,0,A,2019_01,centertosmalldist,61
1,11,0,0,A,2019_01,smalldist,859
2,11,0,0,A,2019_02,centertosmalldist,62
3,11,0,0,A,2019_02,smalldist,834
4,11,0,0,A,2019_03,centertosmalldist,71
...,...,...,...,...,...,...,...
646508,2014,2,5,T,2021_07,centertosmalldist,167
646509,2014,2,5,T,2021_08,centertosmalldist,67
646510,2014,2,5,T,2021_09,centertosmalldist,167
646511,2014,2,5,T,2021_BM,centertosmalldist,53


,ID,day_type,day_part,mode,date_range,group,OVOL,DVOL,BVOL
0,11.0000,0,0,A,2019_01,centertosmalldist,53.0000,61.0000,114.0000
1,11.0000,0,0,A,2019_01,smalldist,852.0000,859.0000,"1,711.0000"
2,11.0000,0,0,A,2019_02,centertosmalldist,54.0000,62.0000,116.0000
3,11.0000,0,0,A,2019_02,smalldist,823.0000,834.0000,"1,657.0000"
4,11.0000,0,0,A,2019_03,centertosmalldist,60.0000,71.0000,131.0000
...,...,...,...,...,...,...,...,...,...
669600,"2,014.0000",2,5,K,2021_03,centertosmalldist,0.0000,12.0000,12.0000
669601,"2,014.0000",2,5,T,2021_01,centertosmalldist,0.0000,44.0000,44.0000
669602,"2,014.0000",2,5,T,2021_07,centertosmalldist,0.0000,167.0000,167.0000
669603,"2,014.0000",2,5,T,2021_08,centertosmalldist,0.0000,67.0000,67.0000


In [84]:
# import date ranges
dfDateRanges = pd.read_csv(os.path.join(intermediate_folder,'dateranges.csv'))
dfDateRanges = dfDateRanges.drop(columns='NUMREC')
dfDateRanges

,date_range,year,season
0,2019_01,2019,01
1,2019_02,2019,02
2,2019_03,2019,03
3,2019_04,2019,04
4,2019_05,2019,05
5,2019_06,2019,06
6,2019_07,2019,07
7,2019_08,2019,08
8,2019_09,2019,09
9,2019_10,2019,10


In [72]:
# move to JAVASCRIPT
#dSeasonToMonths = [
#    ['01', 1, 'M'],
#    ['02', 2, 'M'],
#    ['03', 3, 'M'],
#    ['04', 4, 'M'],
#    ['05', 5, 'M'],
#    ['06', 6, 'M'],
#    ['07', 7, 'M'],
#    ['08', 8, 'M'],
#    ['09', 9, 'M'],
#    ['10',10, 'M'],
#    ['11',11, 'M'],
#    ['12',12, 'M'],
#    ['YR', 1, 'A'],
#    ['YR', 2, 'A'],
#    ['YR', 3, 'A'],
#    ['YR', 4, 'A'],
#    ['YR', 5, 'A'],
#    ['YR', 6, 'A'],
#    ['YR', 7, 'A'],
#    ['YR', 8, 'A'],
#    ['YR', 9, 'A'],
#    ['YR',10, 'A'],
#    ['YR',11, 'A'],
#    ['YR',12, 'A'],
#    ['BM', 4, 'B'],
#    ['BM', 5, 'B'],
#    ['BM', 6, 'B'],
#    ['BM', 7, 'B'],
#    ['BM', 8, 'B'],
#    ['BM', 9, 'B'],
#    ['BM',10, 'B']
#]
#
#dfSeasonToMonths = pd.DataFrame(dSeasonToMonths, columns = ['season','month','seasongroup'])
#display(dfSeasonToMonths)
#
#dDayPartToHours = [
#    [0, 0,'D'], # daily
#    [0, 0,'H'], # hourly --- daily over 24 hours
#    [0, 1,'H'],
#    [0, 2,'H'],
#    [0, 3,'H'],
#    [0, 4,'H'],
#    [0, 5,'H'],
#    [0, 6,'H'],
#    [0, 7,'H'],
#    [0, 8,'H'],
#    [0, 9,'H'],
#    [0,10,'H'],
#    [0,11,'H'],
#    [0,12,'H'],
#    [0,13,'H'],
#    [0,14,'H'],
#    [0,15,'H'],
#    [0,16,'H'],
#    [0,17,'H'],
#    [0,18,'H'],
#    [0,19,'H'],
#    [0,20,'H'],
#    [0,21,'H'],
#    [0,22,'H'],
#    [0,23,'H'],
#    [1, 0,'P'],
#    [1, 1,'P'],
#    [1, 2,'P'],
#    [1, 3,'P'],
#    [1, 4,'P'],
#    [1, 5,'P'],
#    [2, 6,'P'],
#    [2, 7,'P'],
#    [2, 8,'P'],
#    [3, 9,'P'],
#    [3,10,'P'],
#    [3,11,'P'],
#    [3,12,'P'],
#    [3,13,'P'],
#    [3,14,'P'],
#    [4,15,'P'],
#    [4,16,'P'],
#    [4,17,'P'],
#    [5,18,'P'],
#    [5,19,'P'],
#    [5,20,'P'],
#    [5,21,'P'],
#    [5,22,'P'],
#    [5,23,'P']
#]
#
#dfDayPartToHours = pd.DataFrame(dDayPartToHours, columns = ['day_part','hour','daypartgroup'])
#display(dfDayPartToHours)

In [73]:
#dfDateRangesWithMonthBegin = pd.DataFrame.merge(dfDateRanges, dfSeasonToMonths, on='season', how='left')
#dfDateRangesWithMonthBegin['day'] = 1
#dfDateRangesWithMonthBegin['MonthBegin'] = pd.to_datetime(dfDateRangesWithMonthBegin[['year', 'month', 'day']])
#
#from datetime import datetime as dt
#import time
#
#def toYearFraction(date):
#    def sinceEpoch(date): # returns seconds since epoch
#        return time.mktime(date.timetuple())
#    s = sinceEpoch
#
#    year = date.year
#    startOfThisYear = dt(year=year, month=1, day=1)
#    startOfNextYear = dt(year=year+1, month=1, day=1)
#
#    yearElapsed = s(date) - s(startOfThisYear)
#    yearDuration = s(startOfNextYear) - s(startOfThisYear)
#    fraction = yearElapsed/yearDuration
#
#    return date.year + fraction
#
#
#dfDateRangesWithMonthBegin['decimaldate'] = dfDateRangesWithMonthBegin['MonthBegin'].apply(toYearFraction)
#display(dfDateRangesWithMonthBegin)
#
#dfDateRangesWithMonthBegin = dfDateRangesWithMonthBegin[['date_range','year','month','decimaldate','seasongroup']]
#display(dfDateRangesWithMonthBegin)
#
#dfDateRangesWithMonthBegin = pd.DataFrame.merge(dfDateRangesWithMonthBegin, dfDayPartToHours, how='cross')
#
#dfDateRangesWithMonthBegin

In [64]:
# DELETE Season Groups until needed -- Only use Monthly and Period
#dfDateRangesWithMonthBegin = dfDateRangesWithMonthBegin[(dfDateRangesWithMonthBegin['seasongroup' ]=='Monthly') & (dfDateRangesWithMonthBegin['daypartgroup']=='Periods')]
#dfDateRangesWithMonthBegin

In [94]:
dfPlotData_Mon = pd.DataFrame.merge (dfODbyID, dfDateRanges, on=('date_range'))
#dfPlotData = dfPlotData.rename(columns={'OVOL':'O','DVOL':'D','BVOL':'B','day_type':'dt','day_part':'dp','date_range':'dr','mode':'md','group':'g','year':'y','month':'m','decimaldate':'dd','seasongroup':'sg','hour':'h','daypartgroup':'dpg'})

dfPlotData_Mon = dfPlotData_Mon.rename(columns={'OVOL':'O','DVOL':'D','BVOL':'B','day_type':'dt','day_part':'dp','date_range':'dr','mode':'md','group':'g','year':'y'})
dfPlotData_Mon.loc[dfPlotData_Mon['g']=='smalldist'        , 'g'] = 'sd'
dfPlotData_Mon.loc[dfPlotData_Mon['g']=='centertosmalldist', 'g'] = 'ct'

# only use months
dfPlotData_Mon = dfPlotData_Mon[~dfPlotData_Mon['season'].isin(['YR','BM'])]
dfPlotData_Mon['m'] = dfPlotData_Mon['season'].astype(int)
dfPlotData_Mon = dfPlotData_Mon.drop(columns='season')

dfPlotData_Mon['ID'] = dfPlotData_Mon['ID'].astype(int)
dfPlotData_Mon['O' ] = dfPlotData_Mon['O' ].astype(int)
dfPlotData_Mon['D' ] = dfPlotData_Mon['D' ].astype(int)
dfPlotData_Mon['B' ] = dfPlotData_Mon['B' ].astype(int)
dfPlotData_Mon

,ID,dt,dp,md,dr,g,O,D,B,y,m
0,11,0,0,A,2019_01,ct,53,61,114,2019,1
1,11,0,0,A,2019_01,sd,852,859,1711,2019,1
2,11,0,0,K,2019_01,sd,9,12,21,2019,1
3,11,0,0,P,2019_01,sd,601,651,1252,2019,1
4,11,0,0,T,2019_01,ct,27,59,86,2019,1
...,...,...,...,...,...,...,...,...,...,...,...
637436,2011,1,1,K,2021_12,ct,0,12,12,2021,12
637437,2011,1,4,T,2021_12,ct,0,29,29,2021,12
637438,2011,2,3,K,2021_12,ct,0,136,136,2021,12
637439,2012,2,2,K,2021_12,ct,0,54,54,2021,12


In [96]:
dfIDs = dfPlotData_Mon.groupby(['ID'],as_index=False).agg(RECNUM=('B','count'))
dfIDs['ID'] = dfIDs['ID'].astype('int')
display(dfIDs)


for index, row in dfIDs.iterrows():
    print(row['ID'], row['RECNUM'])

    dfPlotData_Mon[dfPlotData_Mon['ID']==row['ID']].to_json(os.path.join(results_folder,'plotdata_monthly_' + str(row['ID']).zfill(4) + '.json'),orient='records')

,ID,RECNUM
0,11,3404
1,21,3444
2,31,4812
3,32,5242
4,41,3814
...,...,...
133,2010,2870
134,2011,2551
135,2012,2782
136,2013,2880


11 3404
21 3444
31 4812
32 5242
41 3814
51 3338
61 2627
62 2328
81 3399
91 4099
101 5143
102 4857
111 4742
112 4110
121 5740
122 5487
131 4864
132 4520
133 3903
141 2604
161 3139
181 5278
182 4166
191 4052
192 4330
201 4349
202 5729
211 5626
212 4843
221 5610
222 4522
231 4533
232 4590
241 3002
261 4155
271 3946
281 4386
291 4730
292 5724
301 4610
302 5577
311 5070
312 5668
313 5717
314 3637
321 5760
331 5748
332 5489
333 5730
334 5189
335 4800
336 5738
337 4291
338 4919
339 4475
341 5734
351 5465
352 5456
361 3439
362 2913
381 3293
391 4010
401 4588
411 4723
412 5701
421 5451
422 5524
423 5587
424 3055
431 4221
432 5090
433 5115
434 4831
441 5632
442 4300
451 5697
452 4398
461 4647
471 3217
481 3445
491 3513
501 2468
511 3421
512 3732
521 2006
522 3812
523 3442
531 4734
532 3828
541 5484
542 5612
551 4182
561 4476
562 4561
571 4470
572 3787
581 5229
582 5189
583 5669
584 4459
585 4544
591 5737
592 5383
593 4899
594 5477
601 3965
602 4679
603 4228
611 3532
612 4018
621 4058
631 2997
64

In [98]:
# TOD JSONs

dfPlotData_TOD = pd.DataFrame.merge (dfODbyID, dfDateRanges, on=('date_range'))
#dfPlotData = dfPlotData.rename(columns={'OVOL':'O','DVOL':'D','BVOL':'B','day_type':'dt','day_part':'dp','date_range':'dr','mode':'md','group':'g','year':'y','month':'m','decimaldate':'dd','seasongroup':'sg','hour':'h','daypartgroup':'dpg'})

dfPlotData_TOD = dfPlotData_TOD.rename(columns={'OVOL':'O','DVOL':'D','BVOL':'B','day_type':'dt','day_part':'dp','date_range':'dr','mode':'md','group':'g','year':'y','season':'s'})
dfPlotData_TOD.loc[dfPlotData_TOD['g']=='smalldist'        , 'g'] = 'sd'
dfPlotData_TOD.loc[dfPlotData_TOD['g']=='centertosmalldist', 'g'] = 'ct'

 # remove all day values for charting
dfPlotData_TOD = dfPlotData_TOD[~dfPlotData_TOD['dp'].isin([0])]

dfPlotData_TOD['ID'] = dfPlotData_TOD['ID'].astype(int)
dfPlotData_TOD['O' ] = dfPlotData_TOD['O' ].astype(int)
dfPlotData_TOD['D' ] = dfPlotData_TOD['D' ].astype(int)
dfPlotData_TOD['B' ] = dfPlotData_TOD['B' ].astype(int)
display(dfPlotData_TOD)

dfIDs = dfPlotData_TOD.groupby(['ID'],as_index=False).agg(RECNUM=('B','count'))
dfIDs['ID'] = dfIDs['ID'].astype('int')
display(dfIDs)


for index, row in dfIDs.iterrows():
    print(row['ID'], row['RECNUM'])

    dfPlotData_TOD[dfPlotData_TOD['ID']==row['ID']].to_json(os.path.join(results_folder,'plotdata_tod_' + str(row['ID']).zfill(4) + '.json'),orient='records')

,ID,dt,dp,md,dr,g,O,D,B,y,s
6,11,0,1,A,2019_01,ct,1,3,4,2019,01
7,11,0,1,A,2019_01,sd,53,53,106,2019,01
8,11,0,1,P,2019_01,sd,70,70,140,2019,01
9,11,0,1,T,2019_01,sd,334,262,596,2019,01
10,11,0,2,A,2019_01,ct,7,3,10,2019,01
...,...,...,...,...,...,...,...,...,...,...,...
669598,701,1,2,T,2021_YR,sd,0,2,2,2021,YR
669599,701,2,1,A,2021_YR,sd,0,2,2,2021,YR
669601,721,0,2,T,2021_YR,ct,0,2,2,2021,YR
669603,721,1,2,T,2021_YR,ct,0,3,3,2021,YR


,ID,RECNUM
0,11,3281
1,21,3332
2,31,4609
3,32,5038
4,41,3661
...,...,...
133,2010,2750
134,2011,2455
135,2012,2662
136,2013,2760


11 3281
21 3332
31 4609
32 5038
41 3661
51 3238
61 2529
62 2262
81 3267
91 3931
101 4913
102 4651
111 4534
112 3935
121 5500
122 5262
131 4655
132 4334
133 3750
141 2516
161 3017
181 5036
182 3982
191 3905
192 4146
201 4164
202 5489
211 5388
212 4627
221 5369
222 4328
231 4335
232 4391
241 2931
261 3971
271 3775
281 4199
291 4521
292 5484
301 4413
302 5336
311 4831
312 5428
313 5477
314 3470
321 5520
331 5508
332 5248
333 5490
334 4967
335 4567
336 5498
337 4109
338 4698
339 4264
341 5494
351 5228
352 5232
361 3318
362 2818
381 3166
391 3839
401 4389
411 4508
412 5461
421 5223
422 5279
423 5345
424 2929
431 4050
432 4856
433 4859
434 4593
441 5390
442 4124
451 5457
452 4200
461 4429
471 3095
481 3312
491 3372
501 2357
511 3326
512 3609
521 1937
522 3670
523 3326
531 4525
532 3681
541 5254
542 5371
551 4015
561 4283
562 4364
571 4278
572 3616
581 4995
582 4978
583 5432
584 4265
585 4339
591 5497
592 5172
593 4681
594 5254
601 3812
602 4492
603 4044
611 3415
612 3850
621 3891
631 2901
64